In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../src/')
import utils as ut
import plotter
import wave as w
import pickle

In [2]:
def build_assay_wave(id_, r0, D, beta, use_fft, gamma=0.1, cutoff=1):
    """
    It created a wave object with a good tuning of the parameters.
    The object has to be run.
    """
    
    k = r0**2*(beta - gamma)/D
    print(k)
    dx = ut.lin_from_two_log_points(r0, 0.5, 0.1, 100, 0.6)
    dt = ut.dt_from_cfl(0.03, D, dx)
    nbins = int(ut.lin_from_two_loglog_points(r0, 0.5, 150, 100, 15000)/dx)

    p = w.Vwave_pars(id_, tot_time=ut.lin_from_two_loglog_points(r0, 0.75, 2000, 100, 30000), dt=dt, dx=dx, 
                     n_x_bins=nbins, 
                     M=5, r0=r0, D_coef=D, beta=beta, alpha=0.0, gamma=gamma,
                     Nh=10**10, N0=10**8, cutoff=cutoff,
                     t_burn=ut.lin_from_two_loglog_points(r0, 0.5, 3000, 100, 200000), 
                     t_burn_cutoff=ut.lin_from_two_loglog_points(r0, 0.5, 400, 100, 8000), 
                     back_width_fract=ut.lin_from_two_log_points(k, 2.5, 0.2, 10**6, 2),
                     traj_step=int(20/dt), check_step=int(0/dt), traj_after_burn=True, verbose=True, use_fft=use_fft)
    return w.Vwave(p)

In [3]:
def build_assay_wave_discr(id_, r0, D, beta, dx, use_fft, gamma=0.1, cutoff=1):
    """
    It created a wave object with a good tuning of the parameters.
    The object has to be run.
    """
    
    k = r0**2*(beta - gamma)/D
    print(k)
    dx_aux = ut.lin_from_two_log_points(r0, 0.5, 0.1, 100, 0.6)
    dt = ut.dt_from_cfl(0.03, D, dx_aux)
    nbins = int(ut.lin_from_two_loglog_points(r0, 0.5, 150, 100, 15000)/dx)

    p = w.Vwave_pars(id_, tot_time=ut.lin_from_two_loglog_points(r0, 0.75, 2000, 100, 30000), dt=dt, dx=dx, 
                     n_x_bins=nbins, 
                     M=5, r0=r0, D_coef=D, beta=beta, alpha=0.0, gamma=gamma,
                     Nh=10**10, N0=10**8, cutoff=cutoff,
                     t_burn=ut.lin_from_two_loglog_points(r0, 0.5, 3000, 100, 200000), 
                     t_burn_cutoff=ut.lin_from_two_loglog_points(r0, 0.5, 400, 100, 8000), 
                     back_width_fract=ut.lin_from_two_log_points(k, 2.5, 0.2, 10**6, 2),
                     traj_step=int(20/dt), check_step=int(0/dt), traj_after_burn=True, verbose=True, use_fft=use_fft)
    return w.Vwave(p)

## Continuous case test

In [23]:
r0 = 1
D = 2e-3
beta = 0.12
wave_fft = build_assay_wave(0, r0, D, beta, True, gamma=0.1, cutoff=1)
wave_fft.run()

9.999999999999995
proc. 0 WARNING: dx is not small compared to r0. dx/r0= 0.1654120103239064
 

Process 0:   0%|          | 0/18376 [00:00<?, ?it/s]

In [24]:
print('averaged fft speed:', np.mean(wave_fft.traj.speed(20)))
print('averaged fft speed:', np.mean(wave_fft.traj.nvirus))

averaged fft speed: 0.012289484172567882
averaged fft speed: 23748604.61195119


In [19]:
wave = build_assay_wave(0, r0, D, beta, False, gamma=0.1, cutoff=1)
wave.run()

9.999999999999995
proc. 0 WARNING: dx is not small compared to r0. dx/r0= 0.1654120103239064
 

Process 0:   0%|          | 0/18376 [00:00<?, ?it/s]

In [22]:
print('averaged fft speed:', np.mean(wave.traj.speed(20)))
print('averaged fft speed:', np.mean(wave_fft.traj.nvirus))

averaged fft speed: 0.012289484172567882
averaged fft speed: 23748634.77204462


## Discrete case test

In [32]:
r0 = 5
D = 2e-3
beta = 0.12
wave_fft = build_assay_wave_discr(0, r0, D, beta, 1, False, gamma=0.1, cutoff=1)
wave_fft.run()

249.9999999999999
proc. 0 WARNING: dx is not small compared to r0. dx/r0= 0.2
 

Process 0:   0%|          | 0/16108 [00:00<?, ?it/s]

In [33]:
print('averaged fft speed:', np.mean(wave_fft.traj.speed(20)))
print('averaged fft speed:', np.mean(wave_fft.traj.nvirus))

averaged fft speed: 0.012367966096939801
averaged fft speed: 4628695.964952155


In [34]:
wave = build_assay_wave_discr(0, r0, D, beta, 1, True, gamma=0.1, cutoff=1)
wave.run()

249.9999999999999
proc. 0 WARNING: dx is not small compared to r0. dx/r0= 0.2
 

Process 0:   0%|          | 0/16108 [00:00<?, ?it/s]

In [35]:
print('averaged fft speed:', np.mean(wave.traj.speed(20)))
print('averaged fft speed:', np.mean(wave_fft.traj.nvirus))

averaged fft speed: 0.012452549451706107
averaged fft speed: 5385144.834642412
